### CIFAR10 Dataset 및 DataLoader 생성. Simple CNN 모델 생성

In [ ]:
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch.utils.data import random_split

#전체 6만개 데이터 중, 5만개는 학습 데이터용. 이를 다시 학습과 검증용으로 split , 1만개는 테스트 데이터용
train_dataset = CIFAR10(root='./data', train=True, download=True, transform=ToTensor())
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=ToTensor())

tr_size = int(0.85 * len(train_dataset))
val_size = len(train_dataset) - tr_size
tr_dataset, val_dataset = random_split(train_dataset, [tr_size, val_size])
print('tr:', len(tr_dataset), 'valid:', len(val_dataset))

tr_loader = DataLoader(tr_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# from torchinfo import summary

NUM_INPUT_CHANNELS = 3

class SimpleCNNWithBN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        #padding 1로 conv 적용 후 출력 면적 사이즈를 입력 면적 사이즈와 동일하게 유지.
        self.conv_block_1 = self.create_convbn_block(first_channels=3, middle_channels=32, last_channels=32)

        #out_channels이 64인 2개의 Conv2d. stride=1이 기본값, padding='same'은 version 1.8에서 소개됨.
        self.conv_block_2 = self.create_convbn_block(first_channels=32, middle_channels=64, last_channels=64)

        # filter갯수 128개인 Conv Layer 2개 적용 후 Max Pooling 적용.
        self.conv_block_3 = self.create_convbn_block(first_channels=64, middle_channels=128, last_channels=128)

        # GAP 및 최종 Classifier Layer
        self.classifier_block = nn.Sequential(
            nn.AdaptiveAvgPool2d(output_size=(1, 1)),
            nn.Flatten(),
            nn.Linear(in_features=128, out_features=num_classes)
        )

    def create_convbn_block(self, first_channels, middle_channels, last_channels):
        conv_bn_block = nn.Sequential(
            nn.Conv2d(in_channels=first_channels, out_channels=middle_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(middle_channels),
            nn.ReLU(),
            nn.Conv2d(in_channels=middle_channels, out_channels=last_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(last_channels),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        return conv_bn_block

    def forward(self, x):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.conv_block_3(x)
        x = self.classifier_block(x)

        return x

def create_do_classifier_block(first_features, second_features, first_dos, second_dos, num_classes=10):
    return nn.Sequential(
            nn.Flatten(),
            nn.Dropout(p=first_dos),
            nn.Linear(in_features=first_features, out_features=second_features),
            nn.ReLU(),
            nn.Dropout(p=second_dos),
            nn.Linear(in_features=second_features, out_features=num_classes),
        )


### 다양한 Learning Rate Scheduler
* pytorch는 nn.optim.lr_scheduler에서 다양한 유형의 Learning Rate Scheduler 클래스를 지원
* 검증 지표를 입력 받고, 조건에 맞춰서 epochs시 마다 Learning Rate를 변경하는 Scheduler와 검증 지표를 입력 받지 않고 epochs 반복 시 특정 패턴으로 Learning Rate를 변경하는 Scheduler 가 있음.
* ReduceLROnPlateur는 (주로) 검증 데이터를 기반의 평가 지표(loss등)를 사용하여 수행. 검증 평가 수치를 epoch시마다 모니터링 하면서 지속적으로 나빠지면 Learning rate를 training oop의 새로운 epoch시에 변경함.
* StepLR, CycleLR, CosineAnnealingLR 등은 설정된 패턴에 따라 Learning Rate를 주기적으로 변경

### Learning Rate Scheduler 생성 및 적용.
* Learning Rate Scheduler는 초기 생성 시 Optimizer 객체를 요구하는 데, 이는 optimzer에 의해서 관리되는 learning 관련 파라미터를 바로 접근하기 위해서임. Learning rate 변경은 Optimizer를 통해서 이뤄져야 함.
* LR Scheduler는 scheduler.step() 와 같이 step() 메소드를 호출하여 learning rate의 Scheduler를 변경.
* scheduler.step()은 epoch 마다 호출이 되어야 하며(배치 단위 아님) optimizer.step()이후에 호출 
* Learning rate정보는 optimizer에서 optimizer.param_groups[0]['lr'] 로도 볼수 있고, scheduler.get_last_lr()[0] 또는 scheduler.get_lr()[0] 로 볼 수 있음.
* scheduler.get_last_lr()[0]는 scheduler.step()으로 적용된 learning rate를, scheduler.get_lr()[0]는 scheduler.step()를 호출 시 다음에 적용될 learning rate를 반환.  

In [ ]:
import torch
import torch.nn as nn
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau

NUM_CLASSES = 10

sample_model = SimpleCNNWithBN(num_classes=NUM_CLASSES)
optimizer = Adam(sample_model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=1, gamma=0.5)

print('optimizer lr:', optimizer.param_groups[0]['lr'])#optimizer.param_groups[0].keys()
print('get_last_lr():', scheduler.get_last_lr()[0])
print('get_lr():', scheduler.get_lr()[0])

scheduler.step()

print('optimizer lr:', optimizer.param_groups[0]['lr'])#optimizer.param_groups[0].keys()
print('get_last_lr():', scheduler.get_last_lr()[0])
print('get_lr():', scheduler.get_lr()[0])

### Learning Rate Scheduler를 적용하도록 Trainer 클래스를 변경
* scheduler.step()은 epoch 마다 호출이 되어야 하며(배치 단위 아님) optimizer.step()이후에 호출 
* ReduceLROnPlateur 는 (주로) Validation epoch에 적용되며 scheduler.step(val_loss)와 같이 step()메소드의 인자로 평가 수치 정보를 입력해 줌.
* 다른 LR Scheduler는 (주로) Train epoch에 적용되며, scheduler.step()을 호출하여 수행됨.

In [ ]:
from tqdm import tqdm
import torch.nn.functional as F

class Trainer:
    def __init__(self, model, loss_fn, optimizer, train_loader, val_loader, scheduler=None, device=None):
        self.model = model.to(device)
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        # scheduler 추가
        self.scheduler = scheduler
        self.device = device
        # 현재 learning rate 변수 추가
        self.current_lr = self.optimizer.param_groups[0]['lr']

    def train_epoch(self, epoch):
        self.model.train()

        # running 평균 loss 계산.
        accu_loss = 0.0
        running_avg_loss = 0.0
        # 정확도, 정확도 계산을 위한 전체 건수 및 누적 정확건수
        num_total = 0.0
        accu_num_correct = 0.0
        accuracy = 0.0
        # tqdm으로 실시간 training loop 진행 상황 시각화
        with tqdm(total=len(self.train_loader), desc=f"Epoch {epoch+1} [Training..]", leave=True) as progress_bar:
            for batch_idx, (inputs, targets) in enumerate(self.train_loader):
                # 반드시 to(self.device). to(device) 아님.
                inputs = inputs.to(self.device)
                targets = targets.to(self.device)

                # Forward pass
                outputs = self.model(inputs)
                loss = self.loss_fn(outputs, targets)

                # Backward pass
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                # batch 반복 시 마다 누적  loss를 구하고 이를 batch 횟수로 나눠서 running 평균 loss 구함.
                accu_loss += loss.item()
                running_avg_loss = accu_loss /(batch_idx + 1)

                # accuracy metric 계산
                # outputs 출력 예측 class값과 targets값 일치 건수 구하고
                num_correct = (outputs.argmax(-1) == targets).sum().item()
                # 배치별 누적 전체 건수와 누적 전체 num_correct 건수로 accuracy 계산  
                num_total += inputs.shape[0]
                accu_num_correct += num_correct
                accuracy = accu_num_correct / num_total

                #tqdm progress_bar에 진행 상황 및 running 평균 loss와 정확도 표시
                progress_bar.update(1)
                if batch_idx % 20 == 0 or (batch_idx + 1) == progress_bar.total:  # 20 batch 횟수마다 또는 맨 마지막 batch에서 update
                    progress_bar.set_postfix({"Loss": running_avg_loss,
                                              "Accuracy": accuracy})

        if (self.scheduler is not None) and (not isinstance(self.scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau)):
            self.scheduler.step()
            self.current_lr = self.scheduler.get_last_lr()[0]
            print("scheduler step() call")

        return running_avg_loss, accuracy

    def validate_epoch(self, epoch):
        if not self.val_loader:
            return None

        self.model.eval()

        # running 평균 loss 계산.
        accu_loss = 0.0
        running_avg_loss = 0.0
        # 정확도, 정확도 계산을 위한 전체 건수 및 누적 정확건수
        num_total = 0.0
        accu_num_correct = 0.0
        accuracy = 0.0
        
        with tqdm(total=len(self.val_loader), desc=f"Epoch {epoch+1} [Validating]", leave=True) as progress_bar:
            with torch.no_grad():
                for batch_idx, (inputs, targets) in enumerate(self.val_loader):
                    inputs = inputs.to(self.device)
                    targets = targets.to(self.device)

                    outputs = self.model(inputs)

                    loss = self.loss_fn(outputs, targets)
                    # batch 반복 시 마다 누적  loss를 구하고 이를 batch 횟수로 나눠서 running 평균 loss 구함.
                    accu_loss += loss.item()
                    running_avg_loss = accu_loss /(batch_idx + 1)

                    # accuracy metric 계산
                    # outputs 출력 예측 class값과 targets값 일치 건수 구하고
                    num_correct = (outputs.argmax(-1) == targets).sum().item()
                    # 배치별 누적 전체 건수와 누적 전체 num_correct 건수로 accuracy 계산  
                    num_total += inputs.shape[0]
                    accu_num_correct += num_correct
                    accuracy = accu_num_correct / num_total

                    #tqdm progress_bar에 진행 상황 및 running 평균 loss와 정확도 표시
                    progress_bar.update(1)
                    if batch_idx % 20 == 0 or (batch_idx + 1) == progress_bar.total:  # 20 batch 횟수마다 또는 맨 마지막 batch에서 update
                        progress_bar.set_postfix({"Loss": running_avg_loss,
                                                  "Accuracy":accuracy})
        # scheduler에 검증 데이터 기반에서 epoch레벨로 계산된 loss를 입력해줌.
        if (self.scheduler is not None) and isinstance(self.scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
            self.scheduler.step(running_avg_loss)
            self.current_lr = self.scheduler.get_last_lr()[0]
            print("scheduler step(evaluation_value) call")

        return running_avg_loss, accuracy

    def fit(self, epochs):
        # epoch 시마다 학습/검증 결과를 기록하는 history dict 생성. learning rate 추가
        history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'lr': []}
        for epoch in range(epochs):
            train_loss, train_acc = self.train_epoch(epoch)
            val_loss, val_acc = self.validate_epoch(epoch)
            print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f} Train Accuracy: {train_acc:.4f}",
                  f", Val Loss: {val_loss:.4f} Val Accuracy: {val_acc:.4f}" if val_loss is not None else "",
                  f", Current lr:{self.current_lr:.6f}")
            # epoch 시마다 학습/검증 결과를 기록. learning rate 추가
            history['train_loss'].append(train_loss); history['train_acc'].append(train_acc)
            history['val_loss'].append(val_loss); history['val_acc'].append(val_acc)
            history['lr'].append(self.current_lr)

        return history

    # 학습이 완료된 모델을 return
    def get_trained_model(self):
        return self.model

#### Learning Rate Scheduler가 적용된 Trainer를 이용하여 모델 학습
* ReduceLROnPlateur의 주요 생성 인자
  * mode: 'min'또는 'max'이며 min은 값이 작아질 수록 개선 되는 지표(예: loss), max는 값이 커질 수록 개선되는 지표(예: 정확도)
  * factor: learning rate 변경 적용 시 곱해지는 scale 값.  new_lr = lr * factor. 기본은 0.1
  * patience: 학습률 감소 결정 전 지표가 더 이상 나아지지 않는 epoch 횟수. patience 수를 넘길 때 까지 지표 개선되지 않으면 감소
  * threshold: patience에서 무시해도 될 정도의 작은 지표 변경값
  * min_lr: 더 이상 감소하지 않을 최소 학습률
  * cooldown: 한번 학습률 감소 후 다시 정상적으로 감소 판별 로직을 수행하기 전까지 기다리는 epoch 횟수

In [ ]:
import torch
import torch.nn as nn
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

NUM_INPUT_CHANNELS = 3
NUM_CLASSES = 10

model = SimpleCNNWithBN(num_classes=NUM_CLASSES)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
scheduler = ReduceLROnPlateau(
            optimizer=optimizer, mode='min', factor=0.5, patience=2, threshold=0.01, min_lr=0.00001)

trainer = Trainer(model=model, loss_fn=loss_fn, optimizer=optimizer,
       train_loader=tr_loader, val_loader=val_loader, scheduler=scheduler, device=device)
# 학습 및 평가
history = trainer.fit(30)

In [ ]:
class Predictor:
    def __init__(self, model, device):
        self.model = model.to(device)
        self.device = device

    def evaluate(self, loader):
        self.model.eval()
        eval_metric = 0.0
        # 정확도 계산을 위한 전체 건수 및 누적 정확건수
        num_total = 0.0
        accu_num_correct = 0.0

        with tqdm(total=len(loader), desc=f"[Evaluating]", leave=True) as progress_bar:
            with torch.no_grad():
                for batch_idx, (inputs, targets) in enumerate(loader):
                    inputs = inputs.to(self.device)
                    targets = targets.to(self.device)
                    pred = self.model(inputs)

                    # 정확도 계산을 위해 누적 전체 건수와 누적 전체 num_correct 건수 계산  
                    num_correct = (pred.argmax(-1) == targets).sum().item()
                    num_total += inputs.shape[0]
                    accu_num_correct += num_correct
                    eval_metric = accu_num_correct / num_total

                    progress_bar.update(1)
                    if batch_idx % 20 == 0 or (batch_idx + 1) == progress_bar.total:
                        progress_bar.set_postfix({"Accuracy": eval_metric})
        
        return eval_metric

    def predict_proba(self, inputs):
        self.model.eval()
        with torch.no_grad():
            inputs = inputs.to(self.device)
            outputs = self.model(inputs)
            #예측값을 반환하므로 targets은 필요 없음.
            #targets = targets.to(self.device)
            pred_proba = F.softmax(outputs, dim=-1) #또는 dim=1

        return pred_proba

    def predict(self, inputs):
        pred_proba = self.predict_proba(inputs)
        pred_class = torch.argmax(pred_proba, dim=-1)

        return pred_class

In [ ]:
trained_model = trainer.get_trained_model()

# 학습데이터와 동일하게 정규화된 데이터를 입력해야 함.
# test_dataset = CIFAR10(root='./data', train=False, download=True, transform=ToTensor())
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

predictor = Predictor(model=trained_model, device=device)
eval_metric = predictor.evaluate(test_loader)
print(f'test dataset evaluation:{eval_metric:.4f}')

### 다양한 Learning Rate Scheduler
* StepLR, CycleLR, CosineAnnealingLR, CosineAnnealingWarmRestarts 등의 동작방식 이해 

#### StepLR
* 특정 Step시 마다 Learning Rate를 감소 시킴

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# 모델 정의
model = nn.Linear(10, 1)
optimizer = optim.Adam(model.parameters(), lr=0.1)
# 학습률을 epoch마다 저장.
lr_history = []
# StepLR 생성
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)

# training loop 시뮬레이션
print("Epoch\tLearning Rate")
for epoch in range(20):
    print(f"{epoch+1}\t{scheduler.get_last_lr()[0]:.6f}")
    lr_history.append(scheduler.get_last_lr()[0])
    scheduler.step()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(4, 3)), 
x_label = [ index+1 for index in range(len(lr_history))]
plt.plot(x_label, lr_history, marker='o')
plt.xlabel('Epochs Number (Index starts from 1)')
plt.ylabel('Learning Rate')

plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

def get_lr_history(scheduler, epochs=30, verbose=False):
  lr_history = []

  # Simulate training loop
  if verbose:
    print("Epoch\tLearning Rate")
  for epoch in range(epochs):
    if verbose:
      print(f"{epoch+1}\t{scheduler.get_last_lr()[0]:.6f}")
    lr_history.append(scheduler.get_last_lr()[0])
    scheduler.step()
  return lr_history

def draw_lr_history(lr_history, figsize=(6, 3)):
    plt.figure(figsize=figsize)
    x_label = [ index+1 for index in range(len(lr_history))]
    plt.plot(x_label, lr_history,  marker='o')
    
    plt.xlabel('Epochs Number (Index starts from 1)')
    plt.ylabel('Learning Rate')
    
    plt.grid(True)
    plt.show()

model = nn.Linear(10, 1)
optimizer = optim.Adam(model.parameters(), lr=0.1)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)

lr_history = get_lr_history(scheduler=scheduler)
draw_lr_history(lr_history)


### MultiStepLR
* milestones에 지정된 step별로 learning rate를 조정

In [ ]:
model = nn.Linear(10, 1)
optimizer = optim.SGD(model.parameters(), lr=0.1)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5, 8, 12], gamma=0.5)

lr_history = get_lr_history(scheduler=scheduler) #verbose=True
draw_lr_history(lr_history)

#### CycleLR
* 학습률이 최소에서 최대로 반복적으로 변함. 학습률(eta)값이 증가/감소를 반복 수행.
* base_lr: 최소 학습률
* max_lr: 최대 학습률
* step_size_up: 최대 학습률에 이르는 epoch 횟수
* step_size_down: 최소 학습률에 이르는 epoch 횟수
* mode: 학습률 cycle의 형태. triangular, triangular2, exp_range가 있음
    * triangular: 삼각파형. 반복 시에도 최대/최소 학습률이 동일
    * triangular2: 삼각파형인데, 반복 시마다 최대 학습률이 절반으로 감소함
    * exp_range: 지수감소형이며, gamma 값 factor레벨로 epoch시마다 학습률이 감소
    * gamma: exp_range에서 적용될 학습률 감소 factor값

In [ ]:
model = nn.Linear(10, 1)
optimizer = optim.Adam(model.parameters(), lr=0.001)
# base_lr은 optimizer의 lr가 동일하게 설정.
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.01,
                                            step_size_up=5,  mode='exp_range', gamma=0.9)
# scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.01,
#                                             step_size_up=10, mode="exp_range", gamma=0.9)
lr_history = get_lr_history(scheduler=scheduler, verbose=True)
draw_lr_history(lr_history)

#### CosineAnnealing
* Cosine선분 형태로 학습률을 최대에서 최소로 점진적으로 감소하는데, 이 Cosine 선분을 주기적으로 반복함.
    * 최대 학습률 -> 최소 학습률로 Cosine 선분 형태로 감소. 다시 최대 학습률-> 최소 학습률로 Cosine 반복
* T_max: 최대->최소 학습률에 이르는 epochs 횟수.
* eta_min: 최소 학습률

In [ ]:
model = nn.Linear(10, 1)
optimizer = optim.SGD(model.parameters(), lr=0.001)
# 시작은 optimizer의 최초 learning rate, eta_min은 가장 최소 learning rate
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0.0001)
lr_history = get_lr_history(scheduler=scheduler, epochs=60)
draw_lr_history(lr_history, figsize=(12, 4))

#### CosineAnnealingWarmRestarts
* CosineAnnealing과 유사하지만, cosine iteration 마다 주기적으로 epoch가 늘어나며 최소 eta에서 최대 eta로 학습률을 바로 높임
* T_0: 첫번째 Cosine iteration 시 최소 학습률(eta)까지 떨어지는 epochs 횟수
* T_mult: Cosine iteration을 반복 시 마다 늘어나는 epochs 횟수 factor.
    * T_0가 10, T_mult가 2라면 첫번째 Cos iter'는 10회에서 최소 eta,  두번째 Cos iter'는 30회에서 최소 eta
* eta_min: 최소 eta(학습률)


In [ ]:
model = nn.Linear(10, 1)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=0.0001) #T_0를 20으로 변경
lr_history = get_lr_history(scheduler=scheduler, epochs=150)
draw_lr_history(lr_history, figsize=(12, 4))

#### CosineAnnealingWarmRestarts을 CNN 모델에 적용

In [ ]:
import torch
import torch.nn as nn
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

NUM_INPUT_CHANNELS = 3
NUM_CLASSES = 10

model = SimpleCNNWithBN(num_classes=NUM_CLASSES)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=0.0001) # T_max=10로 변경 후 테스트
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min=0.0001)
# scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=0.01, step_size_up=10, mode='exp_range', gamma=0.9)
trainer = Trainer(model=model, loss_fn=loss_fn, optimizer=optimizer,
       train_loader=tr_loader, val_loader=val_loader, scheduler=scheduler, device=device)
# 학습 및 평가
history = trainer.fit(60) # fit을 30에서 60으로 변경. 

In [ ]:
trained_model = trainer.get_trained_model()

# 학습데이터와 동일하게 정규화된 데이터를 입력해야 함.
# test_dataset = CIFAR10(root='./data', train=False, download=True, transform=ToTensor())
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

predictor = Predictor(model=trained_model, device=device)
eval_metric = predictor.evaluate(test_loader)
print(f'test dataset evaluation:{eval_metric:.4f}')

### Weight Decay
* 가중치 규제(Weight Regularization)를 적용. 값을 키우면 오버피팅을 개선할 수 있지만, 언더 피팅을 할 수 있음. 값이 너무 작으면 오버피팅 개선 효과가 작아짐.
* Optimizer의 생성 인자에 weight_decay를 설정해줌. 보통 1e-2 ~ 1e-3 사이 값을 적용
* weight decay를 적용 시 Adam 보다는 AdamW에 적용하는 것이 효과적. 

In [ ]:
import torch
import torch.nn as nn
from torch.optim import SGD, Adam, AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

NUM_INPUT_CHANNELS = 3
NUM_CLASSES = 10

model = SimpleCNNWithBN(num_classes=NUM_CLASSES)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = AdamW(model.parameters(), lr=0.001, weight_decay=1e-2) #weight_decay=1e-3 
loss_fn = nn.CrossEntropyLoss()

scheduler = ReduceLROnPlateau(
            optimizer=optimizer, mode='min', factor=0.5, patience=2, threshold=0.01, min_lr=0.00001)

trainer = Trainer(model=model, loss_fn=loss_fn, optimizer=optimizer,
       train_loader=tr_loader, val_loader=val_loader, scheduler=scheduler, device=device)

history = trainer.fit(30)

In [ ]:
trained_model = trainer.get_trained_model()

# 학습데이터와 동일하게 정규화된 데이터를 입력해야 함.
# test_dataset = CIFAR10(root='./data', train=False, download=True, transform=ToTensor())
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

predictor = Predictor(model=trained_model, device=device)
eval_metric = predictor.evaluate(test_loader)
print(f'test dataset evaluation:{eval_metric:.4f}')